## what did I learn from v1?

Even though it was instructed to avoid it, Goldfang (still just find the name funny) made huge trades anyway. It invested ~40% in Nike because it thought it was undervalued, and hey, maybe it is / was. But it will take months or years to prove out if this kind of investing is really successful or just lucky, and by then the place will be saturated by bots that are doing something similar because it's just too easy.

The basic loop I built was to look at stocks one at a time, and ignore almost all other information. I chose randomly from a list of 200 S&P 500 stocks that ChatGPT made. Maybe there's a better way to choose which stocks to look at: which ones have news associated with them, and ask Goldfang if it's worth trading on this news.

I built a placeholder for feeding back open-ended data into the model, but haven't decided how to implement that. I don't think there's anything I could use as an _effective_ oracle right now. It would be sweet if I could feed those queries into a Bing Chat API or Perplexity API, have it do whatever searches it needs to do, and receive a paragraph back of text answering the question in a way that's probably correct.

#### things to try
- [ ] margin trading / short selling: https://alpaca.markets/docs/trading/margin-and-shorting/
- [ ] trade only on newsy stocks
- [ ] what happens if I just use ChatGPT as the oracle for now?
- [ ] put in more macroeconomic data
- [ ] token optimization: delete unnecessary fields, shorten numbers, etc
- [ ] token optimization: detect when token overflow will / might happen, and remove content
- [ ] fix broken API calls by moving to https://data.nasdaq.com/

## Short selling
read https://alpaca.markets/docs/trading/margin-and-shorting/

## Get all news since last run / in last 24 hours, and examing those stocks
using https://alpaca.markets/docs/market-data/news/

In [1]:
import requests
import json
with open('alpaca-keys.json') as f:
    alpaca = json.loads(f.read())
    
def get_news_summary(symbol="", limit=10):
    headers = {
        'Apca-Api-Key-Id': alpaca['key'],
        'Apca-Api-Secret-Key': alpaca['secret']
    }
    if len(symbol) > 0:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news?symbols={symbol}', headers=headers).json()
    else:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news', headers=headers).json()
    news_str = ""
    for item in j['news'][:limit]:
        news_str += f"{item['created_at']}: {item['headline']}"
        if len(item['summary'].strip()) > 0:
            news_str += ' – summary: '+item['summary'].replace('\n', ' ').strip() + '\n'
        else:
            news_str += '\n'
    return news_str
# print(get_news_summary('AAPL'))

In [2]:
from collections import defaultdict

def summarize_news_list(news_list, include_body=False):
    if include_body:
        raise Exception("currently not sure how to get the body without scraping it")
    news_str = ""
    for item in news_list:
        news_str += f"{item['created_at']}: {item['headline']}"
        if len(item['summary'].strip()) > 0:
            news_str += ' – summary: '+item['summary'].replace('\n', ' ').strip() + '\n'
        else:
            news_str += '\n'
    return news_str

def get_news_dict(limit=50, symbols=[]):
    headers = {
        'Apca-Api-Key-Id': alpaca['key'],
        'Apca-Api-Secret-Key': alpaca['secret']
    }
    if len(symbols) == 0:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news?limit={limit}', headers=headers).json()
    else:
        j = requests.get(f'https://data.alpaca.markets/v1beta1/news?limit={limit}&symbols={",".join(symbols)}', headers=headers).json()
    sym_to_news = defaultdict(list)
    for item in j['news']:
        for symbol in item['symbols']:
            sym_to_news[symbol].append(item)
    return sym_to_news
# news_dict = get_news_dict()

In [3]:
# for symbol, news_list in news_dict.items():
#     print(symbol)
#     print(summarize_news_list(news_list))
#     break

In [4]:
# created_dates = [item['created_at'] for news_list in news_dict.values() for item in news_list]

In [5]:
# sorted(created_dates)[0], sorted(created_dates)[-1]

So the range for these last 50 news articles is about a day. Seems good to me!

## Use ChatGPT as its own oracle
extract its previous queries and see what happens when fed back into itself with the correct system prompt.

In [6]:
from openai_utils import query_openai

In [7]:
system_oracle = '''
You are Libra Anker, a market analyst for. You work for Fantasia Goldfang, a financial advisor for higher-income working professionals. 

Every day, your Fantasia will query you for information relevant to asset allocations. Your answers, which go into the final report and 
decision-making, are always crisp, clear, concise, and factual.
'''

prompt_oracle = '''
The following is a query the ticker symbol {symbol}:
{gpt_query}

Dated: {date_str}
'''

In [28]:
with open('notes.json') as f:
    notes = json.loads(f.read())

In [29]:
def extract_query(decision):
    try:
        return decision.split('Information requests:')[1].split('Verdict')[0].strip()
    except:
        return '?'

In [30]:
from datetime import datetime
queries = {}
for symbol, note_list in notes.items():
    note = note_list[-1]
    # print(symbol, extract_query(note['note']))
    query = extract_query(note['note'])
    if 'news' in query:
        print('skipping query for news:', query)
        continue
    queries[symbol] = { 'symbol': symbol, 'query': query, 'date_str': note['date'] }

skipping query for news: Please provide the latest quarterly report for NVDA and any news on upcoming product launches.
skipping query for news: Please check the latest news on General Mills, Inc. and any upcoming changes.
skipping query for news: "latest news on Monster Beverage Corporation"
skipping query for news: Search for recent news surrounding Yum! Brands, Inc. to get an idea of the future outlook for the company. Additionally, look into the company's competition and industry trends.
skipping query for news: Search for news articles and analyst reports on Exelon Corporation's current operations, any major upcoming projects, and their expected impact on the company's stock price. Additionally, retrieve historical trends and the future outlook of the utility sector.
skipping query for news: Search for recent news about Kimberly-Clark Corp.'s management and plans for future growth.
skipping query for news: 1) Recent news (if available)
2) Dividend history of Ecolab, Inc.
3) Indust

In [31]:
len(queries)

38

Maybe I should tell Goldfang not to query for news (at least, until I've got this hooked up to a live oracle).

In [32]:
import random
print(random.choice(list(queries.values())))

{'symbol': 'RMD', 'query': '- Market trends and forecasts for the respiratory medical equipment and software industry\n- Any significant changes in the competitive landscape for ResMed Inc., including new entrants to the industry and changes in market share\n- Potential future catalysts or disruptors for the industry or ResMed Inc. specifically', 'date_str': '2023-03-24 15:21:02.841990'}


In [34]:
queries

{'WEC': {'symbol': 'WEC',
  'query': "1. Latest earnings call notes \n2. Competitors' sales growth figures \n3. Dividend history and payout ratio",
  'date_str': '2023-03-24 14:59:27.472868'},
 'PG': {'symbol': 'PG',
  'query': "Please provide the latest updates on the company's marketing strategies and product innovations. Also, provide an overview of the competition in the consumer goods industry, with a focus on the companies that directly compete with Procter & Gamble.",
  'date_str': '2023-03-24 15:11:19.924480'},
 'AIG': {'symbol': 'AIG',
  'query': "Please provide a detailed analysis of AIG's recent business performance and future growth prospects. Also, provide the latest update on the interest rate environment and inflation trends.",
  'date_str': '2023-03-24 15:11:34.405238'},
 'HLT': {'symbol': 'HLT',
  'query': '1. The recent initiatives taken by the company to boost profitability.\n2. The industry and market trends with respect to hospitality companies.',
  'date_str': '20

In [33]:
for symbol in list(queries.keys())[:3]:
    query = queries[symbol]
    prompt = prompt_oracle.format(symbol=query['symbol'], gpt_query=query['query'], date_str=query['date_str'])
    print(prompt)
    oracle_response = query_openai(prompt, system_oracle, model="gpt-3.5-turbo", debug=False, max_tokens=300)
    print(oracle_response)
    print("="*45)


The following is a query the ticker symbol WEC:
1. Latest earnings call notes 
2. Competitors' sales growth figures 
3. Dividend history and payout ratio

Dated: 2023-03-24 14:59:27.472868

Dear Fantasia,

Regarding your query for the ticker symbol WEC, here is the information you requested:

1. Latest earnings call notes: 

WEC Energy Group held its latest earnings call on February 16, 2023. During the call, the company reported earnings per share of $0.90, which beat analysts' estimates of $0.85. The company's revenue for the quarter was $2.1 billion, compared to the consensus estimate of $2.04 billion.

2. Competitors' sales growth figures: 

According to the latest available data, WEC Energy Group's competitors' sales growth figures are as follows:
- Dominion Energy: 4.8%
- Exelon: 1.7%
- Duke Energy: 2.4%
- Southern Company: 5.5%

3. Dividend history and payout ratio:

WEC Energy Group has a dividend history of consistently paying dividends to its shareholders. Currently, the com

example:

```
The following is a query the ticker symbol NVDA:
Please gather further data on NVDA's dividend history, market competition, and supply chain risks.

Dated: 2023-03-24 15:29:03.251108

Dear Fantasia,

Upon conducting a thorough analysis of NVDA, here are the requested details on its dividend history, market competition, and supply chain risks:

Dividend History:
NVDA has a relatively short dividend history as it only began paying dividends in 2012. Since then, the company has steadily increased its dividend payout, with a current yield of 0.14%. However, it's essential to note that NVDA's primary focus is on growth, and dividend payments are not its top priority.

Market Competition:
NVDA operates in a highly competitive market, with key competitors including Advanced Micro Devices (AMD), Intel (INTC), and Qualcomm (QCOM). While NVDA has managed to gain a significant market share in the gaming and data center markets, competitors are continually innovating and releasing new products, which could impact NVDA's market position.

Supply Chain Risks:
NVDA's supply chain is relatively complex, with the company relying on multiple suppliers for essential components. As such, there is always a risk of supply chain disruptions, particularly in times of global economic turbulence or geopolitical tensions. Any disruptions could result in production delays, which could ultimately impact NVDA's financial performance.

I hope this information helps in your assessment of NVDA.

Regards,
Libra Anker
```

## token optimizing a previous prompt

## token optimizing a too-long prompt

## Loop 2

In [14]:
system_vanilla = '''
You are Fantasia Goldfang, a financial advisor for higher-income working professionals. Your clients want to invest their portfolio in stocks, 
and your job is to take in financial information and give investment advice. 

Every day, your client Montgomery Frostam will query you, and you will decide what to do with his portfolio assets. Your strategy tends to 
center around a company's fundamental value rather than responding to noisy market fluctuations. You should aim to be well-diversified across 
companies and industries: no one company should be a significant fraction of Mr. Frostam's portfolio. When you buy, don't spend more than 5% 
of the total value of the portfolio unless it is truly an exceptional value proposition.

Mr. Frostam's query will include the company's name, a description, the price of its stock, possibly some recent news, and relevant company 
financial statistics. You will also receive information about Montgomery Frostam's portfolio, including how much stock of the current company 
he might hold. You will then respond with the following structured response:

Analysis: (your analysis of the current health of the company, and what its stock's true value might be)
Risk level: (low, medium, or high)
Information requests: (a succinct prompt for your market analyst, Libra Anker, to search additional data for your decision, phrased in the form of a command)
Verdict: (buy, hold, pass, or sell) X (if a buy or sell, X is how many to buy/sell)
'''

In [15]:
prompt_template = '''
Company: {company}
Ask price: {ask_price}
Bid price: {bid_price}
full trading data (Alpaca Market API, get_stock_snapshot):
{trading_data}

last annual income statement for {company} (Financial Modeling Prep API):
{quarterly_data}

previous annual income statement for {company} (Financial Modeling Prep API):
{quarterly_data_1yr}

last 12 months of the Consumer Price Index (Financial Modeling Prep API):
{cpi_data}

stock price change data for {company} (Financial Modeling Prep API):
{stock_change_data}

Recent news (Benzinga news service):
{news}

Analysis by Libra Anker:
{analysis}

Montgomery Frostam's portfolio 
{company} stock owned: {stock_owned} (${stock_value} value)
Portfolio cash available: ${cash}
Portfolio total value: ${total_value}
'''

In [16]:
import fmp_utils
import alpaca_utils

In [17]:
import os
from datetime import datetime
def write_log(s):
    try:
        with open('log.md') as f:
            txt = f.read()
    except:
        txt = ""
    with open('_log.md', 'w') as f:
        f.write(txt + '\n' + s)
    os.rename('_log.md', 'log.md')

In [18]:
def write_notes(symbol, decision):
    try:
        with open('notes.json') as f:
            j = json.loads(f.read())
    except:
        j = {}
    if symbol not in j:
        j[symbol] = []
    j[symbol].append({ 'note': decision, 'date': str(datetime.now()) })
    with open('_notes.json', 'w') as f:
        f.write(json.dumps(j, indent='  '))
    os.rename('_notes.json', 'notes.json')
    
def get_notes(symbol):
    try:
        with open('notes.json') as f:
            j = json.loads(f.read())
    except:
        j = {}
    notes = ""
    if symbol in j:
        for item in j[symbol]:
            notes += f"{item['date']}\n{item['note'].strip()}"
    return notes
# print(get_notes('NVDA'))

In [19]:
def get_analysis(decision):
    analysis = decision.split('Risk level:')[0].replace('Analysis:', '')
    return analysis

In [20]:
def get_verdict(decision, symbol):
    # should return { 'verdict': 'buy|sell|hold|pass', 'amount': X }
    decision = decision.lower()
    verdict_par = decision.split('verdict:')[1].strip()
    verdict = verdict_par.split(' ')[0]
    for char in set('!.'):
        verdict = verdict.replace(char, '')
    if verdict not in ('buy', 'sell', 'hold', 'pass'):
        raise Exception(verdict + ' not valid!')
    if verdict in ('buy', 'sell'):
        amount = verdict_par.split(' ')[1]
    else:
        amount = 0
    return { 'verdict': verdict, 'amount': int(amount), 'symbol': symbol }

def handle_verdict(verdict, portfolio):
    spending_cap = 1000
    
    if verdict['verdict'] in ('hold', 'pass'):
        return
    symbol = get_symbol(verdict['guild'])
    if verdict['verdict'] == 'sell' and verdict['amount'] > portfolio[symbol]['held']:
        print(f"can't sell {verdict['amount']} {symbol}, only own {portfolio[symbol]['held']}")
    quote = get_quote(symbol)
    if verdict['amount'] * quote > spending_cap:
        verdict['amount'] = int(spending_cap / quote)
        print('capping trade at', verdict['amount'], symbol)
    trade(symbol, verdict['verdict'], verdict['amount'])
    return update_portfolio(portfolio, verdict)

In [21]:
def count_tokens(s, divisor=3):
    # placeholder until I make / get something better, maybe, unless this is good enough
    return int(len(s) / divisor)

In [22]:
portfolio = alpaca_utils.get_portfolio()
cpi_data = [{'date': '2023-02-01', 'value': 301.648}, {'date': '2023-01-01', 'value': 300.536}, {'date': '2022-12-01', 'value': 298.99}, {'date': '2022-11-01', 'value': 298.598}, {'date': '2022-10-01', 'value': 297.987}, {'date': '2022-09-01', 'value': 296.539}, {'date': '2022-08-01', 'value': 295.32}, {'date': '2022-07-01', 'value': 294.628}, {'date': '2022-06-01', 'value': 294.728}, {'date': '2022-05-01', 'value': 291.268}, {'date': '2022-04-01', 'value': 288.611}, {'date': '2022-03-01', 'value': 287.472}]
write_log('# ' + str(datetime.now()))
news_dict = get_news_dict() # get symbols with recent news
symbols_with_news = list(news_dict.keys())
random.shuffle(symbols_with_news)

for symbol in symbols_with_news:
    write_log('\n## ' + symbol)
    news_list = news_dict[symbol]
    
    # get financial information
    asset_data = alpaca_utils.get_asset(symbol)
    try:
        snapshot_data = alpaca_utils.get_snapshot(symbol)
    except:
        print(symbol, "snapshot not found")
        continue
    del snapshot_data[symbol].daily_bar
    del snapshot_data[symbol].previous_daily_bar
    quote = snapshot_data[symbol].latest_quote
    quarterly = fmp_utils.get_financials(symbol)
    quarterly_1yr = fmp_utils.get_financials(symbol, 1)
    stock_change = fmp_utils.get_simple_stock_changes(symbol)

    # get news and analysis
    oracle_request = queries[symbol] if symbol in queries else None
    if oracle_request:
        oracle_prompt = prompt_oracle.format(symbol=oracle_request['symbol'], gpt_query=oracle_request['query'], date_str=oracle_request['date_str'])
        oracle_response = query_openai(oracle_prompt, system_oracle, model="gpt-3.5-turbo", debug=False, max_tokens=300)
    else:
        oracle_response = '(none)'
        print('no oracle request', symbol)
        continue
    # news = summarize_news_list(news_list)
    news = alpaca_utils.get_news_summary(symbol) # gets more news
    notes = get_notes(symbol)
    notes = get_analysis(get_notes(symbol))
    if len(notes) == 0:
        print('no notes!', symbol)
        continue
    # notes = '(none)' # need to remove info request, and just keep verdict
    stock_owned = portfolio[symbol]['held'] if symbol in portfolio else 0
    prompt = prompt_template.format(
        company=asset_data.name, news=news, 
        ask_price=quote.ask_price, 
        bid_price=quote.bid_price, 
        trading_data=snapshot_data[symbol].json(), 
        quarterly_data=quarterly,
        stock_change_data=stock_change, 
        quarterly_data_1yr=quarterly_1yr, 
        cpi_data=str(cpi_data),
        stock_owned=stock_owned, 
        stock_value=stock_owned * float(quote.bid_price), 
        cash=portfolio['cash'], 
        total_value=portfolio['value'], 
        analysis=oracle_response)
    if len(notes) > 0:
        prompt += f"\n\nYour previous notes on {asset_data.name}:\n{notes}"
    print("** tokens:", count_tokens(system_vanilla+prompt))
    try:
        decision = query_openai(prompt, system_vanilla, model="gpt-3.5-turbo", debug=True)
    except Exception as e:
        print(e)
        continue
    write_log(decision)
    write_notes(symbol, decision)
    print(get_verdict(decision, symbol))
    # new_portfolio = handle_verdict(get_verdict(decision, symbol), portfolio)
    print("~"*45)
    # if new_portfolio['cash'] <= 0:
    #     break

no oracle request SHOP
CNNWQ snapshot not found
** tokens: 3178
~ prompt: 
Company: Apple Inc. Common Stock
Ask price: 0.0
Bid price: 0.0
full trading data (Alpaca Market API, get_stock_snapshot):
{"symbol": "AAPL", "latest_trade": {"symbol": "AAPL", "timestamp": "2023-03-24T19:59:58.800099+00:00", "exchange": "V", "price": 160.25, "size": 200.0, "id": 7348, "conditions": ["@"], "tape": "C"}, "latest_quote": {"symbol": "AAPL", "timestamp": "2023-03-24T20:00:00.000522+00:00", "ask_exchange": "V", "ask_price": 0.0, "ask_size": 0.0, "bid_exchange": "V", "bid_price": 0.0, "bid_size": 0.0, "conditions": ["R"], "tape": "C"}, "minute_bar": {"symbol": "AAPL", "timestamp": "2023-03-24T19:59:00+00:00", "open": 160.18, "high": 160.34, "low": 160.09, "close": 160.25, "volume": 24805.0, "trade_count": 229.0, "vwap": 160.18847}}

last annual income statement for Apple Inc. Common Stock (Financial Modeling Prep API):
{'date': '2022-09-24', 'symbol': 'AAPL', 'reportedCurrency': 'USD', 'cik': '00003201

In [23]:
symbol

'SQ'

In [24]:
print(news)

2023-03-24T21:30:06Z: GH Research, Cybin Among Top Psychedelic Movers Of Today
2023-03-23T21:30:06Z: GH Research, Compass Pathways Among Top Psychedelic Movers Of Today
2023-03-22T21:30:06Z: Psychedelic Stock Gainers And Losers From March 22, 2023
2023-03-21T21:30:05Z: Psychedelic Stock Gainers And Losers From March 21, 2023
2023-03-20T21:30:06Z: Psychedelic Stock Gainers And Losers From March 20, 2023
2023-03-17T21:30:06Z: Psychedelic Stock Gainers And Losers From March 17, 2023
2023-03-16T21:30:05Z: Seelos Therapeutics, Compass Pathways Among Top Psychedelic Movers Of Today
2023-03-15T21:30:06Z: Seelos Therapeutics, Cybin Among Top Psychedelic Movers Of Today
2023-03-14T21:30:06Z: Seelos Therapeutics, ATAI Life Sciences Among Top Psychedelic Movers Of Today
2023-03-13T21:30:06Z: ATAI Life Sciences, Compass Pathways Among Top Psychedelic Movers Of Today



Hahaha. Libra just makes up a $114 million quarterly profit. Yikes. And somehow today the quarterly earnings data is no longer available (maybe I was on a trial for those particular endpoints? Boo)

## Making Libra better

I was surprised to see that the oracle, Libra Anker, knew about Silicon Valley Bank collapsing. Looks like it only knew about it because the question mentioned it. I know that LLMs are supposed to be excellent bullshit machines, but when they have correct information they can get much closer to the truth. Right now, though, Libra Anker is a more pure bullshit machine. Let me do a quick analysis to see what information I could pass in her prompt so she could bullshit less. And in general:
- pass data to Libra to maximize the number of questions she'll have information on
- come up with other "advisor" figures who can chime in on certain dimensional axes. How about an advisor for each major industry? For popular sentiment? Etc
- Maybe I could make an advisor that could just query FMP...

In [35]:
from datetime import datetime
queries = defaultdict(list)
for symbol, note_list in notes.items():
    # note = note_list[-1]
    # print(symbol, extract_query(note['note']))
    for note in note_list:
        query = extract_query(note['note'])
        queries[symbol].append({ 'symbol': symbol, 'query': query, 'date_str': note['date'] })

In [43]:
q_str = ''
for symbol, qlist in queries.items():
    # print(symbol)
    for q in qlist:
        # print('\t-'+q['query'])
        q_str += '-'+q['query'].replace('\n', ' ') + '\n'

In [54]:
print(q_str)

-Please gather further data on NVDA's dividend history, market competition, and supply chain risks.
-Please gather further data on NVDA's dividend history, market competition, and supply chain risks.
-Please gather further data on NVDA's dividend history, market competition, and supply chain risks.
-Please provide the latest quarterly report for NVDA and any news on upcoming product launches.
-Please check the latest news on General Mills, Inc. and any upcoming changes.
-1. Latest earnings call notes  2. Competitors' sales growth figures  3. Dividend history and payout ratio
-"latest news on Monster Beverage Corporation"
-Please provide the latest updates on the company's marketing strategies and product innovations. Also, provide an overview of the competition in the consumer goods industry, with a focus on the companies that directly compete with Procter & Gamble.
-Please provide a detailed analysis of AIG's recent business performance and future growth prospects. Also, provide the l

In [50]:
response = query_openai('''The following is a list of queries for a researcher. Please re-order the list by type of request.

'''+q_str, '')

In [51]:
print(response)

Financial Data:
- Dividend history and payout ratio for NVDA
- Latest quarterly report for NVDA
- AIG's recent business performance and future growth prospects
- P/E ratio, dividend yield, and cash flow analysis of ExxonMobil Corporation
- Latest annual report for Phillips 66 and Phillips 66 competitors in the energy and petrochemical industries
- Current P/E ratio, P/B ratio, and dividend yield of Evergy, Inc.
- Cincinnati Financial Corporation Common Stock SWOT analysis
- Comerica Incorporated market trends and competitor analysis
- Latest quarterly report and Consumer Price Index data
- Credit Suisse Group's debt-to-equity ratio and any recent changes in management
- Goldman Sachs Group Inc.'s current debt load and any recent changes to their leadership team
- Latest annual income statement and any other financial data available for ARK Fintech Innovation ETF

News and Updates:
- Latest news on Monster Beverage Corporation
- Latest updates on NVDA's dividend history, market competit

Notes
- cash flow statement is already in annual report
- including quarterly report would be great though
- PE ratio for industry: https://financialmodelingprep.com/api/v4/industry_price_earning_ratio?date=2021-05-07&exchange=NYSE&apikey=db39ae5b594e7fde421d83fdde927c21

Could try something like this prompt for API access, might sometimes work:

```
The following is a high-level description of a financial information API. Please construct a correct URL to query the data necessary to answer questions.
For example, to get Microsoft's latest quarterly report, you would return: /api/v4/financial-reports-json?symbol=MSFT&year=2023&period=Q1&apikey=YOUR_API_KEY

This is the following question to query data for:
"What is the current state of the aviation industry"

Today is March 25, 2023.

--------------------------


## Company Financial Statements
Your API Key : 7648d7883571b6f3daeb64b1a307f29a

Income Statement
Annual income statements Annual JSON: /api/v3/income-statement/AAPL?limit=120&apikey=YOUR_API_KEY
Quarterly income statements Quarter JSON: /api/v3/income-statement/AAPL?period=quarter&limit=400&apikey=YOUR_API_KEY
Income statements by CIK JSON: /api/v3/income-statement/0001318605?period=quarter&apikey=YOUR_API_KEY

Balance Sheet Statement
Annual balance sheet statements Annual JSON: /api/v3/balance-sheet-statement/AAPL?limit=120&apikey=YOUR_API_KEY
Quarterly balance sheet statements Quarter JSON: /api/v3/balance-sheet-statement/AAPL?period=quarter&limit=400&apikey=YOUR_API_KEY
Balance sheet statements by CIK JSON: /api/v3/balance-sheet-statement/0001318605?period=quarter&apikey=YOUR_API_KEY

Cash Flow Statement
Annual cash flow statements Annual JSON: /api/v3/cash-flow-statement/AAPL?limit=120&apikey=YOUR_API_KEY
Quarterly cash flow statements Quarter JSON: /api/v3/cash-flow-statement/AAPL?period=quarter&limit=400&apikey=YOUR_API_KEY
Cash Flow statements by CIK JSON: /api/v3/cash-flow-statement/0001318605?period=quarter&apikey=YOUR_API_KEY

Quarterly Earnings Reports
Quarterly Earnings Reports. Period accepted: Q1/Q2/Q3/Q4 JSON: /api/v4/financial-reports-json?symbol=AAPL&year=2020&period=Q1&apikey=YOUR_API_KEY

Sectors PE Ratio
Average price to earnings ratio for sectors: /api/v4/sector_price_earning_ratio?date=2021-05-07&exchange=NYSE&apikey=YOUR_API_KEY

Industries PE Ratio
Average price to earnings ratio for industries JSON: /api/v4/industry_price_earning_ratio?date=2021-05-07&exchange=NYSE&apikey=YOUR_API_KEY

Stock Market Sectors Performance
Stock Market Sectors Performance. Hourly JSON: /api/v3/sector-performance?apikey=YOUR_API_KEY
Historical sector performance. JSON: /api/v3/historical-sectors-performance?limit=50&apikey=YOUR_API_KEY
```

This did return a working endpoint calling sector-performance, but querying in a follow-up prompt asking it if the data could answer the question led to it saying it could not (and I agree).

For the q: "Please provide information on Credit Suisse Group's debt-to-equity ratio and any recent changes in its management." using the above prompt templates returns this endpoint:

`api/v3/balance-sheet-statement/0001318605?period=quarter&apikey=7648d7883571b6f3daeb64b1a307f29a`

Not sure if that number refers to Credit Suisse, but I wouldn't think so. Endpoint just tells me I've run out of their limit for the day and I should start paying for the API. But using their sample data for that endpoint and feeding it back to GPT after switching out the symbols:

`Based on the data provided, Credit Suisse Group's debt-to-equity ratio is 0.19 as of September 25, 2021. The total debt is 124719000000 USD and the total equity is 63090000000 USD. There has been no recent changes in management.` 

The date is correct, and the numbers it pulled are correct. The calculation is just wrong. And it's actually data for Apple, but that's just a problem with the data. I asked it to write some Python code if it needed to do a calculation, but it didn't. Maybe GPT4 will be more reliable. The calculation is so wrong that I'd hesitate to crap like this in the main prompt I'm passing to Fantasia.

I'll try another query for information, trying again to find one that has a chance of working... this one should be easy, no?

"Please provide the latest annual income statement and any other financial data available for ARK Fintech Innovation ETF."

It returned: `Answer: /api/v3/income-statement/ARKF?limit=120&apikey=7648d7883571b6f3daeb64b1a307f29a`. It seems to like the Answer: format, so I'm incorporating that into the prompt so I'll be able to string match to it. 

Interestingly the endpoint returns nothing, but assuming it would work if my API key worked, I feed it in the Apple dummy data and get:

`Based on the data provided, ARK Fintech Innovation ETF had an annual revenue of $36.58 billion, cost of revenue of $21.30 billion, gross profit of $15.28 billion, operating expenses of $4.39 billion, cost and expenses of $25.69 billion, interest income of $2.84 billion, interest expense of $2.64 billion, depreciation and amortization of $11.28 billion, operating income of $10.89 billion, income before taxes of $10.92 billion, income tax expense of $1.45 billion, and net income of $9.47 billion in 2021. The earnings per share (EPS) was $5.46.`

Hmm... the actual data it pulled is:
```
    "revenue" : 365817000000,
    "costOfRevenue" : 212981000000,
    "costAndExpenses" : 256868000000,
    "operatingExpenses" : 43887000000,
...
```

If those units are dollars, it's 365 B, not 36. Since it's Apple, they probably did do that much revenue, jesus. (Confirmed) 
GPT-4 gets the correct order of magnitude:

```
The latest annual income statement available for ARK Fintech Innovation ETF (ARKF) is for the fiscal year ending on September 25, 2021. Here are the key figures from the income statement:

- Revenue: $365,817,000,000
- Cost of Revenue: $212,981,000,000
...
```

## why isn't the quarterly endpoint working anymore?

In [25]:
with open('fundamentals-keys.json') as f:
    fmp_key = json.loads(f.read())['fmp_key']
def get_quarterly(symbol, idx=0):
    fmp_base_url_v3 = "https://financialmodelingprep.com/api/v3"
    j = requests.get(f"{fmp_base_url_v3}/income-statement/{symbol}?period=quarter&limit={idx+1}&apikey={fmp_key}").json()
    try:
        return j[idx]
    except:
        print(j)
        return {}

In [26]:
get_quarterly('AAPL')

{'Error Message': 'Limit Reach . Please upgrade your plan or visit our documentation for more details at https://site.financialmodelingprep.com/'}


{}

Okay, so I can't do quarterly period. Although it worked yesterday :shrug: